# Preprocessing steps and real-time statistical descriptions

## Importing Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from pyspark.sql import SparkSession
from sklearn.preprocessing import StandardScaler
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.ml.feature import Imputer

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
from pyspark.sql.functions import col

from datetime import datetime

from influxdb_client import InfluxDBClient, Point, WritePrecision
from influxdb_client.client.write_api import SYNCHRONOUS


In [ ]:
# Create a spark session
spark = SparkSession.builder.appName("HAI-Preprocessing").getOrCreate()

## Data Loading

In [ ]:
# Load data
train1 = spark.read.csv("/Users/emmatosato/Documents/UNI_Locale/Erasmus/OST/ost-sm-change-detection/data_analysis/hai/hai-train1.csv", header=True, inferSchema=True)
train2 = spark.read.csv("/Users/emmatosato/Documents/UNI_Locale/Erasmus/OST/ost-sm-change-detection/data_analysis/hai/hai-train2.csv", header=True, inferSchema=True)
test1 = spark.read.csv("/Users/emmatosato/Documents/UNI_Locale/Erasmus/OST/ost-sm-change-detection/data_analysis/hai/haitest1.csv", header=True, inferSchema=True)
test2 = spark.read.csv("/Users/emmatosato/Documents/UNI_Locale/Erasmus/OST/ost-sm-change-detection/data_analysis/hai/hai-test2.csv", header=True, inferSchema=True)
label1 = spark.read.csv("/Users/emmatosato/Documents/UNI_Locale/Erasmus/OST/ost-sm-change-detection/data_analysis/hai/label-test1.csv", header=True, inferSchema=True)
label2 = spark.read.csv("/Users/emmatosato/Documents/UNI_Locale/Erasmus/OST/ost-sm-change-detection/data_analysis/hai/label-test2.csv", header=True, inferSchema=True)

## Data set insights

In [ ]:
# First input
print(f"First input dimensions: ")
print(f"Train: ({train1.count()}, {len(train1.columns)})")
print(f"Test: ({test1.count()}, {len(test1.columns)})")
print(f"Label: ({label1.count()}, {len(label1.columns)})\n")

# Second input
print(f"Second input dimensions: ")
print(f"Train: ({train2.count()}, {len(train2.columns)})")
print(f"Test: ({test2.count()}, {len(test2.columns)})")
print(f"Label: ({label2.count()}, {len(label2.columns)})\n")


In [ ]:
train1.show()

In [ ]:
train1.printSchema()

In [ ]:
train1.describe().show()

In [ ]:
train2.describe().show()

In [ ]:
type_counts = {}
for col_name, col_type in train1.dtypes[1:]:
    if col_type in type_counts:
        type_counts[col_type].append(col_name)
    else:
        type_counts[col_type] = [col_name]

print("Variables by Type:")
for col_type, variables in type_counts.items():
    variable_count = len(variables)
    print(f"- {variable_count} {col_type}{'s' if variable_count > 1 else ''}")


In [ ]:
def calculate_time_interval(dataframe):
    time_interval = dataframe.agg(min("timestamp").alias("start_time"), max("timestamp").alias("end_time")).collect()[0]

    start_time = time_interval["start_time"]
    end_time = time_interval["end_time"]

    print(f"Start Time: {start_time}")
    print(f"End Time: {end_time}")

    duration = end_time - start_time
    print(f"Duration: {duration}\n")

In [ ]:
# List of DataFrames
sets = [train1, test1, train2, test2]
sets_name = ["Train set 1", "Test set 1", "Train set 2", "Test set 2"]

# Calculate time interval and duration for each set
for s, set_name in zip(sets, sets_name):
    print(f"=== Time Interval Analysis for {set_name} ===")
    calculate_time_interval(s)

## Data Preprocessing

### Missing values handling

Printing the NaN values

In [ ]:
def check_null_values(dataframe, name="DataFrame"):
    print(f"{name} - Checking for Null Values:")
    
    # Calculate the sum of null values for each column
    null_counts = dataframe.agg(*[sum(col(c).isNull().cast("int")).alias(c) for c in dataframe.columns]).collect()[0]
    
    # Iterate over columns and print only if there are null values
    found_null_values = False
    for col_ in dataframe.columns:
        null_count = null_counts[col_]
        if null_count > 0:
            print(f"Column '{col_}': {null_count} null values")
            found_null_values = True
    
    # Print a message if no null values are found
    if not found_null_values:
        print(f"No null values found.\n")

In [ ]:
# Check null values for the first input
print("FIRST data input")
check_null_values(train1, name="Train set")
check_null_values(test1, name="Test set")

# Check null values for the second input
print("\n\nSECOND data input")
check_null_values(train2, name="Train set")
check_null_values(test2, name="Test set")

In [ ]:
# Identify numeric columns (excluding timestamp columns)
column_subset = [col_ for col_ in train1.columns if train1.select(col_).dtypes[0][1] != "string" and col_ != "timestamp"]

# Impute NaN values with the median for numeric columns
imputer = Imputer(
    inputCols=column_subset,
    outputCols=[col_ for col_ in column_subset],
    strategy="median"
)

# Fit and transform the DataFrame 
train1 = imputer.fit(train1).transform(train1)

### Duplicated values handling

In [ ]:
def check_duplicates(dataframe, name="DataFrame"):
    found_duplicates = False
    
    print(f"{name} - Checking for Duplicates:")
    
    # Get the count of each row
    row_count = dataframe.count()
    
    # Get the count of distinct rows
    distinct_row_count = dataframe.distinct().count()
    
    if (row_count - distinct_row_count) > 0:
        print(f"{row_count - distinct_row_count} duplicate rows found.")
        found_duplicates = True
    else:
        print(f"No duplicate rows found.\n")

In [ ]:
# Check duplicates for the first input
print("FIRST data input")
check_duplicates(train1, name="Train set")
check_duplicates(test1, name="Test set")

# Check duplicates for the second input
print("\n\nSECOND data input")
check_duplicates(train2, name="Train set")
check_duplicates(test2, name="Test set")


### Normalization

In [ ]:
from pyspark.sql.functions import lit

# Merging train1 and train2
complete_train = train1.union(train2)

# Merging test and labels for the ML part
test_df1 = test1.join(label1, on="timestamp", how="inner")
test_df2 = test2.join(label2, on="timestamp", how="inner")

In [ ]:
print(f"Complete dataset: ({complete_train.count()}, {len(complete_train.columns)})")

In [ ]:
print(f"Complete dataset: ({test_df2.count()}, {len(test_df2.columns)})")

In [ ]:
# To pandas
complete_train_pd = complete_train.toPandas()
test_pd1 = test_df1.toPandas()
test_pd2 = test_df2.toPandas()

In [ ]:
# Create an instance of StandardScaler
scaler = StandardScaler()

# Get the column subset excluding the timestamp column
columns_to_exclude = ["timestamp", "label"]
column_subset_test = [col for col in test_pd1.columns if col not in columns_to_exclude]
column_subset_all = [col for col in complete_train_pd.columns if col != "timestamp"]

# Normalization
scaled_data = scaler.fit_transform(test_pd1[column_subset_test])
scaled_test1 = pd.DataFrame(scaled_data, columns=column_subset_test)
scaled_test1.insert(0, "timestamp", test_pd1["timestamp"])
scaled_test1["label"] = test_pd1["label"]

scaled_data = scaler.fit_transform(test_pd2[column_subset_test])
scaled_test2 = pd.DataFrame(scaled_data, columns=column_subset_test)
scaled_test2.insert(0, "timestamp", test_pd2["timestamp"])
scaled_test2["label"] = test_pd2["label"]

scaled_data = scaler.fit_transform(complete_train_pd[column_subset_all])
scaled_complete = pd.DataFrame(scaled_data, columns=column_subset_all)
scaled_complete.insert(0, "timestamp", complete_train_pd["timestamp"])

### Saving  toCSV

In [ ]:
# Not processed
test_pd1.to_csv("/Users/emmatosato/Documents/UNI_Locale/Erasmus/OST/ost-sm-change-detection/data_analysis/merged_data/test_pd1.csv", index=False, mode='w')
test_pd2.to_csv("/Users/emmatosato/Documents/UNI_Locale/Erasmus/OST/ost-sm-change-detection/data_analysis/merged_data/test_pd2.csv", index=False, mode='w')
complete_train_pd.to_csv("/Users/emmatosato/Documents/UNI_Locale/Erasmus/OST/ost-sm-change-detection/data_analysis/merged_data/complete_pd.csv", index=False, mode='w')

# Processed
scaled_test1.to_csv("/Users/emmatosato/Documents/UNI_Locale/Erasmus/OST/ost-sm-change-detection/data_analysis/preprocessed_data/scaled_test1.csv", index=False, mode='w')
scaled_test2.to_csv("/Users/emmatosato/Documents/UNI_Locale/Erasmus/OST/ost-sm-change-detection/data_analysis/preprocessed_data/scaled_test2.csv", index=False, mode='w')
scaled_complete.to_csv("/Users/emmatosato/Documents/UNI_Locale/Erasmus/OST/ost-sm-change-detection/data_analysis/preprocessed_data/scaled_complete.csv", index=False, mode='w')

## Data Visualization

### Plotting variables distribution

Plotting the distributions of the variables for the different process.
The process flow of the testbed was divided into four primary processes: the boiler process (P1), turbine process (P2), water treatment process (P3), and HIL simulation (P4). The label dataset was marked as 1 only when attack occurred to indicate the presence or absence of an attack.

In [ ]:
def plot_subplots(data, columns_to_plot, title="Train set", height=900):
    fig = make_subplots(rows=len(columns_to_plot), cols=1, subplot_titles=columns_to_plot)

    # Plot each time series in a subplot
    for i, col in enumerate(columns_to_plot, start=1):
        trace = go.Scatter(x=data.index, y=data[col], mode='lines', name=col)
        fig.add_trace(trace, row=i, col=1)

    # Update layout 
    fig.update_layout(
        title_text=title,
        height=height,
        template='plotly_white',  
        legend=dict(y=1.2),
        yaxis=dict(tickmode='linear', tick0=0, dtick=20),
    )
    return fig 

In [ ]:
# Complete train dataset
fig1 = plot_subplots(complete_train_pd, ["P1_FCV01D", "P1_PP04SP", "P1_LCV01D", "P2_24Vdc", "P1_TIT01"], title=" Process 1", height=900)
fig2 = plot_subplots(complete_train_pd, ["P2_SIT01", "P2_VT01", "P2_VTR03"], title="Process 2", height=900)
fig3 = plot_subplots(complete_train_pd, ["P3_FIT01", "P2_VTR03", "P3_LIT01"], title="Process3", height=600)

fig1.show()
fig2.show()
fig3.show()

In [ ]:
# Test1
# fig1 = plot_subplots(test_pd1, ["P1_FCV01D", "P1_PP04SP", "P1_LCV01D", "P2_24Vdc", "P1_TIT01"], title="Test set 1 - Process 1", height=900)
# fig2 = plot_subplots(test_pd1, ["P2_SIT01", "P2_VT01", "P2_VTR03"], title="Test set 1 - Process 2", height=600)
# fig3 = plot_subplots(test_pd1, [ "P3_FIT01", "P3_LIT01"], title= "Test set 1 - Process 3", height=400)

# fig1.show()
# fig2.show()
# fig3.show()

In [ ]:
from dash import Dash, html, dcc, callback, Output, Input
from plotly.subplots import make_subplots

# Define the unique titles for each subplot
subplot_titles_test1 = ["Test set 1 - Process 1", "Test set 1 - Process 2", "Test set 1 - Process 3"]

# Initialize the Dash app
app = Dash(__name__)

# Define the layout of the app
app.layout = html.Div([
    html.H1(children='Test set 1 features distributions', style={'textAlign': 'center'}),
    dcc.Dropdown(
        options=[{'label': title, 'value': title} for title in subplot_titles_test1],
        value=subplot_titles_test1[0],
        id='dropdown-selection'
    ),
    dcc.Graph(id='graph-content')
])

# Define the callback to update the graph based on dropdown selection
@app.callback(
    Output('graph-content', 'figure'),
    [Input('dropdown-selection', 'value')]
)
def update_graph(selected_title):
    # Map selected title to corresponding columns for test_pd1
    title_to_columns_test1 = {
        "Test set 1 - Process 1": ["P1_FCV01D", "P1_PP04SP", "P1_LCV01D", "P2_24Vdc", "P1_TIT01"],
        "Test set 1 - Process 2": ["P2_SIT01", "P2_VT01", "P2_VTR03"],
        "Test set 1 - Process 3": ["P3_FIT01", "P3_LIT01"]
    }

    # Get columns based on selected title
    columns_to_plot_test1 = title_to_columns_test1.get(selected_title, [])

    # Call the plot_subplots function
    fig = plot_subplots(test_pd1, columns_to_plot_test1, title=f'{selected_title}')

    return fig

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)

In [ ]:
# Test2
fig1 = plot_subplots(test_pd2, ["P1_FCV01D", "P1_PP04SP", "P1_LCV01D", "P2_24Vdc", "P1_TIT01"], title="Test set 2 - Process 1", height=900)
fig2 = plot_subplots(test_pd2, ["P2_SIT01", "P2_VT01", "P2_VTR03"], title="Test set 1 - Process 2", height=600)
fig3 = plot_subplots(test_pd2, [ "P3_FIT01", "P3_LIT01"], title= "Test set 1 - Process 3", height=400)

fig1.show()
fig2.show()
fig3.show()

### Attacks Distributions

In [ ]:
test_pd1[(test_pd1['label'] == 1) & (test_pd1['label'].shift() == 0)]

In [ ]:
test_pd1[(test_pd1['label'] == 0) & (test_pd1['label'].shift() == 1)]

We can observe how this variable behave during time (under attack or not).

In [ ]:
changes_df = complete_train_pd[1500:13000]

fig = go.Figure()

trace = go.Scatter(x=changes_df.index, y=complete_train_pd["P1_FCV01D"], mode='lines', name="P1_FCV01D")
fig.add_trace(trace)

# Update layout 
fig.update_layout(
    title_text=f"Changes in P1_FCV01D variable",
    height=500,
    template='plotly_white',  
    legend=dict(y=1.2),
    yaxis=dict(tickmode='linear', tick0=0, dtick=20),
)

# Show the plot
fig.show()